### "How do major world events (e.g., elections, protests, cultural movements) impact the popularity of specific artists or artworks?"

Step 1: Create artistic_influences and artist_trends tables in PostgreSQL

Step 2: Query artist influences from Neo4j

In [ ]:
from neo4j import GraphDatabase
import psycopg2

# Connect to Neo4j
neo4j_driver = GraphDatabase.driver("bolt://localhost:7687", auth=("user", "password"))

# Connect to Postgres
conn = psycopg2.connect(
    dbname="dbname",
    user="user",
    password="password",
    host="host",
    port="port"
)
cursor = conn.cursor()

# Fetch data from Neo4j
query = """
MATCH (h:Artist)-[:INFLUENCED]->(m:Artist)
RETURN h.name AS historical_artist, m.name AS modern_artist;
"""
with neo4j_driver.session() as session:
    results = session.run(query)
    for record in results:
        cursor.execute(
            "INSERT INTO artistic_influences (historical_artist, modern_artist) VALUES (%s, %s)",
            (record["historical_artist"], record["modern_artist"])
        )

# Commit and close connections
conn.commit()
cursor.close()
conn.close()
neo4j_driver.close()

Step 3: Fetch Redis data and import to Python

In [ ]:
import redis

# Connect to Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Connect to Postgres
conn = psycopg2.connect(
    dbname="dbname",
    user="user",
    password="password",
    host="host",
    port="port"
)
cursor = conn.cursor()

# List of artists whose trends we want to migrate to PostgreSQL
artists = ["Pablo Picasso", "Vincent van Gogh"]  # Example list of artists

# Iterate over each artist
for artist in artists:
    # Construct the Redis key for the artist's trends
    key = f"trends:{artist}"
    
    # Fetch all trend data from Redis using zrangebyscore (get all data in the zset)
    trends = r.zrangebyscore(key, "-inf", "+inf", withscores=True)
    
    # Iterate over the retrieved data (timestamp, popularity score)
    for timestamp, popularity in trends:
        # Convert timestamp from Redis (which is in float) to a PostgreSQL-friendly format
        # Redis timestamps are float, PostgreSQL expects a timestamp type or UNIX timestamp.
        
        # If timestamp is in Redis as a UNIX timestamp (float), convert to datetime:
        pg_cursor.execute(
            "INSERT INTO artist_trends (artist_name, event_date, search_popularity) "
            "VALUES (%s, to_timestamp(%s), %s)",
            (artist, timestamp, popularity)
        )

conn.commit()
cursor.close()
conn.close()
r.close()

Step 5: Run final query in PostgreSQL

### Do artists from the same nationality or cultural background trend together?

Step 1: Fetch data from Google Trends API and store in Redis

Step 2: Insert Redis data into PostgreSQL

In [ ]:
# Connect to Postgres
conn = psycopg2.connect(
    dbname="dbname",
    user="user",
    password="password",
    host="host",
    port="port"
)
cursor = conn.cursor()

# Insert artist
artist_name = "Artist Name"
nationality = "Nationality"
cursor.execute("INSERT INTO artists (name, nationality) VALUES (%s, %s)", 
               (artist_name, nationality))
conn.commit()

# Insert trend data
artist_id = cursor.lastrowid
trend_date = "2025-01-01"
trend_value = 10
cursor.execute("INSERT INTO artist_trends (artist_id, date, trend_value) VALUES (%s, %s, %s)", 
               (artist_id, trend_date, trend_value))
conn.commit()

cursor.close()
conn.close()

Step 3: Model relationships in Neo4j

Step 4: Run PostgreSQL query to get list of artists who share the same nationality. Run Cypher queries in Neo4j to find artists with the same nationality.

Step 5: Compute the correlation coefficient / compare trend patterns.

### Does an artist passing away influence their popularity or the popularity of their art styles/previous collaborators?

Step 1: Collect data using Google Trends API and store in Redis

Step 2: Insert trend data into PostgreSQL

In [ ]:
# Connect to Postgres
conn = psycopg2.connect(
    dbname="dbname",
    user="user",
    password="password",
    host="host",
    port="port"
)
cursor = conn.cursor()

# Insert artist data
artist_name = "Vincent van Gogh"
death_date = "1890-07-29"
nationality = "Dutch"
cursor.execute("INSERT INTO artists (name, death_date, nationality) VALUES (%s, %s, %s)", 
               (artist_name, death_date, nationality))
conn.commit()

# Get the artist ID
cursor.execute("SELECT id FROM artists WHERE name = %s", (artist_name,))
artist_id = cursor.fetchone()[0]

# Insert trend data (example trend data)
trend_data = {"2000-01-01": 10, "2000-02-01": 15}
for date, trend_value in trend_data.items():
    cursor.execute("INSERT INTO artist_trends (artist_id, date, trend_value) VALUES (%s, %s, %s)",
                   (artist_id, date, trend_value))
conn.commit()

# Close the connection
cursor.close()
conn.close()

Step 3: Use Neo4j to model relationships and collaborations

Step 4: Query the data to analyze trends and influences

- Trend data before and after artist's death
- Influence on collaborator's popularity
- Trend changes in art styles

Step 5: Use cypher queries in Neo4j to find the influence on collaborators and the trends before and after an artist's death